In [16]:
!pip install peft transformers datasets accelerate bitsandbytes


In [17]:
!pip install peft transformers datasets accelerate bitsandbytes
from huggingface_hub import notebook_login

notebook_login()


In [18]:
from google.colab import files
uploaded = files.upload()

Saving dataset_advocacia.json to dataset_advocacia (1).json


In [19]:
from datasets import load_dataset

# Altere o nome do arquivo se precisar!
dataset = load_dataset("json", data_files="dataset_advocacia.json")

def preprocess(example):
    # Troque os campos conforme o seu dataset!
    return {
        "text": f"Usuário: {example['mensagem_usuario']}\nAdvogado: {example['mensagem_bot']}"
    }

dataset = dataset.map(preprocess)


Map:   0%|          | 0/595 [00:00<?, ? examples/s]

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Pode trocar por outros modelos suportados (pode dar erro de VRAM em modelos grandes na Colab Free)
model_ckpt = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
# Carrega modelo base e tokenizer normalmente (em 8bit)
model = AutoModelForCausalLM.from_pretrained(model_ckpt, device_map="auto", load_in_8bit=True)

from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # troque se der erro
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


In [23]:
# Carrega o tokenizer normalmente
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Função tokenize ajustada para incluir "labels"
def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

# Agora pode mapear normalmente
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/595 [00:00<?, ? examples/s]

In [24]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    per_device_train_batch_size=1,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    output_dir="/content/fine_tuned_advocacia"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
500,5.403400


TrainOutput(global_step=595, training_loss=5.401089657054228, metrics={'train_runtime': 877.8152, 'train_samples_per_second': 0.678, 'train_steps_per_second': 0.678, 'total_flos': 1.300337799462912e+16, 'train_loss': 5.401089657054228, 'epoch': 1.0})

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
500,5.336100


TrainOutput(global_step=595, training_loss=5.3353326364725575, metrics={'train_runtime': 869.753, 'train_samples_per_second': 0.684, 'train_steps_per_second': 0.684, 'total_flos': 1.300337799462912e+16, 'train_loss': 5.3353326364725575, 'epoch': 1.0})

In [26]:
model.save_pretrained("/content/fine_tuned_advocacia")
tokenizer.save_pretrained("/content/fine_tuned_advocacia")

('/content/fine_tuned_advocacia/tokenizer_config.json',
 '/content/fine_tuned_advocacia/special_tokens_map.json',
 '/content/fine_tuned_advocacia/chat_template.jinja',
 '/content/fine_tuned_advocacia/tokenizer.model',
 '/content/fine_tuned_advocacia/added_tokens.json',
 '/content/fine_tuned_advocacia/tokenizer.json')

In [29]:
model.save_pretrained("/content/fine_tuned_advocacia")
tokenizer.save_pretrained("/content/fine_tuned_advocacia")


('/content/fine_tuned_advocacia/tokenizer_config.json',
 '/content/fine_tuned_advocacia/special_tokens_map.json',
 '/content/fine_tuned_advocacia/chat_template.jinja',
 '/content/fine_tuned_advocacia/tokenizer.model',
 '/content/fine_tuned_advocacia/added_tokens.json',
 '/content/fine_tuned_advocacia/tokenizer.json')

In [31]:
model.save_pretrained("/content/fine_tuned_advocacia")

In [32]:
from transformers import pipeline

pipe = pipeline("text-generation", model="/content/fine_tuned_advocacia", tokenizer="/content/fine_tuned_advocacia", device=-1)
prompt = "Boa tarde, preciso de um advogado para direito civil."
result = pipe(prompt, max_new_tokens=100, do_sample=True)
print(result[0]['generated_text'])


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu


KeyboardInterrupt: 

In [33]:
!zip -r fine_tuned_advocacia.zip /content/fine_tuned_advocacia

  adding: content/fine_tuned_advocacia/ (stored 0%)
  adding: content/fine_tuned_advocacia/adapter_config.json (deflated 55%)
  adding: content/fine_tuned_advocacia/runs/ (stored 0%)
  adding: content/fine_tuned_advocacia/runs/Jul27_20-25-09_8000250057ca/ (stored 0%)
  adding: content/fine_tuned_advocacia/runs/Jul27_20-25-09_8000250057ca/events.out.tfevents.1753647914.8000250057ca.5517.0 (deflated 62%)
  adding: content/fine_tuned_advocacia/runs/Jul27_20-29-14_8000250057ca/ (stored 0%)
  adding: content/fine_tuned_advocacia/runs/Jul27_20-29-14_8000250057ca/events.out.tfevents.1753648154.8000250057ca.5517.1 (deflated 60%)
  adding: content/fine_tuned_advocacia/runs/Jul27_20-29-14_8000250057ca/events.out.tfevents.1753649906.8000250057ca.5517.2 (deflated 60%)
  adding: content/fine_tuned_advocacia/checkpoint-595/ (stored 0%)
  adding: content/fine_tuned_advocacia/checkpoint-595/scheduler.pt (deflated 56%)
  adding: content/fine_tuned_advocacia/checkpoint-595/training_args.bin (deflated 52

In [ ]:
!pip install flask flask-ngrok peft transformers accelerate
